In [6]:
# imports and loading
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import sys
sys.path.insert(0, '../functions/')
import interface_GAMS as iG
import conversion_equations as ce
import pickle
import os
from matplotlib.colors import Normalize
import ast
import seaborn as sns

# load in a bunch of potentially useful files
# loading merged log_tpm_file
# merge together log_tpm_df files
log_tpm_df = pd.read_csv('../data/precise_1k/log_tpm.csv', index_col = 0)
starve_log_tpm = pd.read_csv('../data/validation_data_sets/stationary_phase/cleaned_log_tpm_qc.csv', index_col = 0)
to_blank_inds = list(set(log_tpm_df.index) - set(starve_log_tpm.index))
# need to create zero rows for missing values
zeros_data = {col : 0 for col in starve_log_tpm.columns}
zeros_df = pd.DataFrame(zeros_data, index = to_blank_inds)
starve_log_tpm = pd.concat([starve_log_tpm, zeros_df])
starve_log_tpm = starve_log_tpm.loc[log_tpm_df.index]
log_tpm_df = pd.concat([starve_log_tpm, log_tpm_df], axis = 1)

starve_metabs = pd.read_excel('../data/validation_data_sets/stationary_phase/metabolites_data.xlsx', index_col = 0)

pickle_in = open('../data/b_to_gene.pkl', 'rb')
b_to_gene = pickle.load(pickle_in)
pickle_in.close()
gene_to_b = {v : k for k, v in b_to_gene.items()}

flags_df = pd.read_csv('../data/saved_flags_expanded_curated.csv', index_col = 0)
#flags_df = flags_df[flags_df['include']]

TF_flags_df = pd.read_csv('../data/TF_saved_flags_expanded.csv', index_col = 0)

In [7]:
# go case by case, look at expression of genes within said cases' genes
filter_minimum = 0.05 # for each gene, need at least X% of samples to have at least a Y correlation
corr_minimum = 0.5
case_to_mRNA_passed = {}
for iMs_run in set([(row['act_iM'], row['inh_iM']) for _, row in flags_df.iterrows()]):
    clean = [x for x in iMs_run if str(x) != 'nan']
    case = '___'.join([str(iM) for iM in iMs_run]).replace(' ', '_').replace('/', '_')

    keep = []
    for index, row in flags_df.iterrows():
        case2 = '___'.join([str(iM) for iM in [row['act_iM'], row['inh_iM']]]).replace(' ', '_').replace('/', '_')
        if case2 == case:
            keep.append(index)
    bby_flags = flags_df.loc[keep]
    genes = bby_flags.index.to_list()
    
    samples_keep = log_tpm_df.columns[(log_tpm_df.loc[genes].corr() > corr_minimum).sum() > filter_minimum*len(log_tpm_df.columns)].to_list()
    print(case+' : dropping '+str(len(log_tpm_df.columns) - len(samples_keep)))
    
    case_to_mRNA_passed.update({case : samples_keep})

Lysine_T2SS___nan : dropping 7
nan___Purine : dropping 2
Fur-2___Fur-1 : dropping 8
Biotin___nan : dropping 11
Potassium___nan : dropping 9
BasR___nan : dropping 36
Lrp___nan : dropping 4
Sugar_Diacid___nan : dropping 10
Cysteine-2___Cysteine-1 : dropping 7
EvgA___nan : dropping 38
Phosphate-1___nan : dropping 7
Cra___nan : dropping 17
Magnesium___nan : dropping 1
nan___Arginine : dropping 7
Fatty_Acid___nan : dropping 20
Pyruvate-1___nan : dropping 20
Crp-2___nan : dropping 28
Sulfoquinovose___nan : dropping 37
OxyR___nan : dropping 1
Crp-1___nan : dropping 1
DNA_Damage___nan : dropping 5
nan___ArcA : dropping 1
YcjW___nan : dropping 12
FlhDC-2___nan : dropping 21
Crp-2___DhaR : dropping 3
SoxS___nan : dropping 1
nan___Cysteine-1 : dropping 1
Capsule___nan : dropping 10
nan___Fur-1 : dropping 12
Fur-2___nan : dropping 2
nan___Lrp : dropping 2
Glutamine___nan : dropping 29
Maltose___nan : dropping 30
CpxR___nan : dropping 4
nan___Nitrogen : dropping 1
Phosphate-2___nan : dropping 4
Met

In [8]:
# save it off
pickle_out = open('../data/case_to_mRNA_passed.pkl', 'wb')
pickle.dump(case_to_mRNA_passed, pickle_out)
pickle_out.close()